In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import time
import sys
import random
import tensorflow as tf
import numpy as np
import importlib
import argparse
import facenet
import lfw
import h5py
import math
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import data_flow_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops

In [4]:
def main(args):
    network = importlib.import_module(args.model_def)
    image_size = (args.image_size, args.image_size)
    
    subdir = datetime.strftime(datatime.now(), '%Y%m%d-%H%M%S')
    log_dir = os.path.join(os.path.expanduser(args.logs_base_dir), subdir)
    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)
    model_dir = os.path.join(os.path.expanduser(args.models_base_dir), subdir)
    if not os.path.isdir(model_dir):
        os.makedirs(model_dir)
        
    stat_file_name = os.path.join(log_dir, 'stat.h5')
    
    facenet.write_arguments_to_file(args, os.path.join(log_dir, 'arguments.txt'))
    
    src_path,_ = os.path.split(os.path.realpath(__file__))
    facenet.store_revision_info(src_path, log_dir, ' '.join(sys.argv))
    
    np.random.seed(seed=args.seed)
    random.seed(args.seed)
    dataset = facenet.get_dataset(args.data_dir)

    if args.validation_set_split_ratio > 0.0:
        train_set, val_set = facenet.split_dataset(dataset, args.validation_set_split_ratio, args.min_nrof_val_images_per_class, 
                                                   'SPLIT_IMAGES')
    else:
        train_set, val_set = dataset, []    
    
    num_classes = len(train_set)

    print('Model directory: %s' % model_dir)
    print('Log directory: %s' % log_dir)
    
    
    pretrained_model = None
    if args.pretrained_model:
        pretrained_model = os.path.expanduser(args.pretrained_model)
        print('Pre-trained model: %s' % pretrained_model)
        
    if args.lfw_dir:
        print('LFW directory: %s' % args.lfw_dir)
        pairs = lfw.read_pairs(os.path.expanduser(args.lfw_pairs))
        lfw_paths, actual_issame = lfw.get_paths(os.path.expanduser(args.lfw_dir), pairs)
        
    with tf.Graph().as_default():
        tf.set_random_state(args.seed)
        global_step = tf.Variable(0, trainable=False)
        
        image_list, label_list = facenet.get_image_paths_and_labels(train_set)
        assert len(image_list) > 0, 'The training set should not be empty'
        
        val_image_list, val_label_list = facenet.get_image_paths_and_labels(val_set)
        
        labels = ops.convert_to_tensor(label_list, dtype=tf.int32)
        range_size = array_ops.shape(labels)[0]
        index_q = tf.train.range_input_producer(range_size, num_epochs=None, shuffle=True, seed=None, capacity=32)
        index_deq_op = index_q.dequeue_many(args.batch_size * args.epoch_size, 'index_dequeue')
        
        learning_rate_placeholder = tf.placeholder(tf.float32, name='learning_rate')
        batch_size_placeholder = tf.placeholder(tf.int32, name='batch_size')
        phase_train_placeholder = tf.placeholder(tf.bool, name='phase_train')
        image_paths_placeholder = tf.placeholder(tf.string, shape=(None, 1), name='image_paths')
        labels_placeholder = tf.placeholder(tf.int32, shape=(None, 1), name='labels')
        control_placeholder = tf.placeholder(tf.int32, shape=(None, 1), name='control')
        
        num_preprocess_threads = 4
        input_q = data_flow_ops.FIFOQueue(capacity=2000000, shared_name=None, name=None,
                                         dtype=[tf.string, tf.int32, tf.int32], 
                                         shapes=[(1, ), (1, ), (1, )])
        enq_op = input_q.enqueue_many([image_paths_placeholder, labels_placeholder, control_placeholder], name='enq_op')

        image_batch, label_batch = facenet.create_input_pipeline(input_q, image_size, num_preprocess_threads, batch_size_placeholder)
        
        image_batch = tf.identity(image_batch, 'image_batch')
        image_batch = tf.identity(image_batch, 'input')
        label_batch = tf.identity(label_batch, 'label_batch')
        
        print('Number of classes in training set: %d' % nrof_classes)
        print('Number of examples in training set: %d' % len(image_list))
        print('Number of classes in validation set: %d' % len(val_set))
        print('Number of examples in validation set: %d' % len(val_image_list))
        print('Building training graph')        
        
        prelogits, _ = network.inference(image_batch, args.keep_probability, phase_train=phase_train_placeholder, 
                                         bottleneck_layer_size=args.embedding_size, weight_decay=args.weight_decay)
        
        logits = slim.fully_connected(prelogits, num_classes, activation_fn=None, reuse=False, scope='Logits', 
                                     weights_initializer=slim.initializers.xavier_initializer(), 
                                     weights_regularizer=slim.l2_regularizer(args.weight_decay))
        
        embeddings = tf.nn.l2_normalize(prelogits, 1, 1e-10, name='embeddings')
        
        eps = 1e-4
        prelogits_norm = tf.reduce_mean(tf.norm(tf.abs(prelogits) + eps, ord=args.prelogits_norm_p, axis=1))
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, prelogits_norm * args.prelogits_norm_loss_factor)
        
        prelogits_center_loss, _ = facenet.center_loss(prelogits, label_batch, args.center_loss_alfa, num_classes)
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, prelogits_center_loss * args.center_loss_factor)

        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label_batch, logits=logits, name='cross_entropy_per_example')
        cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
        tf.add_to_collection('losses', cross_entropy_mean)

        regularization_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)        
        total_loss = tf.add_n([cross_entropy_mean] + regularization_loss, name='total_loss')

        correct_prediction = tf.cast(tf.equal(tf.argmax(logits, 1), tf.cast(label_batch, tf.int64)), tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        
        learning_rate = tf.train.exponential_decay(learning_rate_placeholder, global_step, 
                                                  args.learning_rate_decay_epochs * args.epoch_size, 
                                                  args.learning_rate_decay_factor, staircase=True,)
        tf.summary.scalar('learning_rate', learning_rate)
        
        train_op = facenet.train(total_loss, global_step, args.optimizer, learning_rate, 
                                args.moving_average_decay, tf.global_variables(), args.log_histograms)
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        